In [57]:
import os
import uuid
import json
import numpy as np
from PIL import Image
from pydantic import BaseModel
from sqlalchemy import create_engine, MetaData

from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import PyPDFLoader
from pprint import pprint

from sentence_transformers import SentenceTransformer

from langchain.utilities import SQLDatabase
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain_ollama import OllamaEmbeddings
from langchain.schema import Document
from langchain.vectorstores import Qdrant

from qdrant_client import QdrantClient, models
from qdrant_client.http.models import PointStruct, VectorParams, Distance

from dotenv import load_dotenv
load_dotenv(override=True)

import warnings
warnings.filterwarnings("ignore")

import yaml

In [58]:
with open("../docs/cm_tools.yaml", "r") as f:
    cm_tools = yaml.safe_load(f)
pprint(cm_tools)

{'cm_tools': [{'content': 'Tool name: RFI (Request for Information) '
                          'Definition: A formal request submitted by a '
                          'contractor or subcontractor to the consultant or '
                          'owner to clarify design details, specifications, or '
                          'site conditions.   Example: When the contractor or '
                          'subcontractor is unsure about design details, '
                          'specifications, or site conditions, they submit an '
                          'RFI. The consultant or owner reviews it, issues a '
                          'response, and the RFI status is updated '
                          'accordingly.\n',
               'id': 'Tool::RFI',
               'metadata': {'process_status': 'document_document.process',
                            'process_status_values': {0: 'Draft',
                                                      1: 'Review',
                             

In [59]:
embeddings = OllamaEmbeddings(model="bge-m3:latest")

In [60]:
points = []
for i, tool in enumerate(cm_tools['cm_tools']):
    id = tool['id']
    title = tool['title']
    content = tool['content']
    metadata = tool['metadata']
    point = PointStruct(
        id=i,
        vector=embeddings.embed_query(content),
        payload={
            "title": title,
            "content": content,
            "metadata": metadata
        }
    )
    points.append(point)


In [61]:
qdrant_client  = QdrantClient(url=os.getenv("QDRANT_URL"))

In [62]:
COL_NAME = "CM-tools-docs"
if not qdrant_client.collection_exists(collection_name=COL_NAME): 
    qdrant_client.create_collection(
        collection_name=COL_NAME,
        vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
        # optional: tune shards, replication_factor, etc.
    )
else:
    print("Collection already created")

Collection already created


In [63]:
qdrant_client.upsert(collection_name=COL_NAME, points=points)

UpdateResult(operation_id=4, status=<UpdateStatus.COMPLETED: 'completed'>)

In [84]:
q = "Does the status of latest RFI is draft?"

In [85]:
resp = qdrant_client.query_points(
    collection_name=COL_NAME,
    query=embeddings.embed_query(q),
    search_params=models.SearchParams(hnsw_ef=128, exact=False),
    limit=5,
)

resp.points

[ScoredPoint(id=0, version=4, score=0.5397983, payload={'title': 'RFI (Request for Information)', 'content': 'Tool name: RFI (Request for Information) Definition: A formal request submitted by a contractor or subcontractor to the consultant or owner to clarify design details, specifications, or site conditions.   Example: When the contractor or subcontractor is unsure about design details, specifications, or site conditions, they submit an RFI. The consultant or owner reviews it, issues a response, and the RFI status is updated accordingly.\n', 'metadata': {'task_decision': 'document_document.status', 'task_decision_values': {'0': 'Invalid', '1': 'Pending', '2': 'In Process', '3': 'Approved', '4': 'Approved as Note', '5': 'Rejected', '6': 'Rejected as Note', '7': 'Deleted'}, 'process_status': 'document_document.process', 'process_status_values': {'0': 'Draft', '1': 'Review', '2': 'Open', '3': 'Waiting For Manager', '4': 'Closed'}, 'related_tables': ['document_document', 'document_templ

# Table Infomation

In [ ]:
with open("../docs/cm_tools.yaml", "r") as f:
    cm_tools = yaml.safe_load(f)
pprint(cm_tools)